In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests, csv
import lxml

# Regex
regex_url = r"(http(s?):\/\/(www)?[A-Za-z0-9-?\/\.]+)"
regex_hastags = r"(#[A-Za-z0-9]{2,})"

# File url
file_url = '../datasets/gdnhealthcare.txt'

# Basic list
csv_header = ['tweetid','date','title']

# Support scripts
%run -i scripts.py

In [2]:
regex_bbc = r"(http(s?):\/\/(www)?[A-Za-z0-9\/\.]+)"
# regex_cbc = r"(http(s?):\/\/(www)?[A-Za-z0-9-?=\s\/\.]+)"
xpath_bbc = ['//div[@class="gs-container"]/div[1]/div/p/text()',
            '',
            '//div[@class="meta__contact-wrap"]/p[1]/text()']

# Regex "&amp"
regex_decode = r"(\&amp)"

In [3]:
frame = pd.read_table(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame.head()

,tweetid,date,title
0,585876545266286592,Wed Apr 08 18:47:08 +0000 2015,What is palliative care like in India? One GP ...
1,585861945535791106,Wed Apr 08 17:49:07 +0000 2015,Most viewed this week: I loved being a midwife...
2,585859917350731777,Wed Apr 08 17:41:03 +0000 2015,How can technology improve mental health waiti...
3,585844465199407104,Wed Apr 08 16:39:39 +0000 2015,In case you missed it: Why the #NHS shouldn’t ...
4,585800529739001856,Wed Apr 08 13:45:04 +0000 2015,India is a stark example of how commodifying h...


In [4]:
frame.shape

(2993, 3)

In [5]:
frame.loc[:,'title'][0]

'What is palliative care like in India? One GP trainee explains all: http://gu.com/p/47a4g/stw'

In [6]:
# -------------------------------------------------
# Split Url & Text
frame['url'] = frame['title'].map(lambda row: get_url(regex_url, row))
frame['title'] = frame['title'].map(split_txt_form_url)

http://gu.com/p/47a4g/stw
http://gu.com/p/46zc4/stw
http://gu.com/p/476ky/stw
http://gu.com/p/477tc/stw
http://gu.com/p/47a4g/stw
http://www.theguardian.com/healthcare-network/2015/apr/07/why-nhs-should-not-hog-limelight-general-election
http://gu.com/p/47a4g/stw
http://gu.com/p/47ajt/stw
http://www.theguardian.com/healthcare-network/2015/apr/02/technology-mental-health-waiting-times
http://gu.com/p/477tc/stw
http://gu.com/p/47a4g/stw
http://gu.com/p/477tc/stw
http://www.vice.com/en
http://gu.com/p/476fa/stw
http://gu.com/p/47a5x/stw
http://gu.com/p/477tc/stw
http://www.theguardian.com/healthcare-network/2015/apr/07/why-nhs-should-not-hog-limelight-general-election
http://gu.com/p/478kc/stw
http://gu.com/p/477tc/stw
http://gu.com/p/477tc
http://gu.com/p/476ky/stw
http://gu.com/p/45445/stw
http://sumall.com/myweek
http://gu.com/p/46hnm/stw
http://gu.com/p/472d5/stw
http://gu.com/p/46gy5/stw
http://gu.com/p/476ky/stw
http://gu.com/p/45yzh/stw
http://gu.com/p/475ch/stw
http://gu.com/p/474

http://bit.ly/1J87nMs
http://bit.ly/1J87nMs
http://www.theguardian.com/healthcare-network/2014/dec/18/how-nhs-organisations-should-use-social-media
http://www.theguardian.com/society/2014/dec/18/labour-nhs-england-christmas-news-blackout
https://witness.theguardian.com/assignment/54919d48e4b0c3fc3f345e0e
http://bit.ly/1BXqeYo
http://t.co/s1UL
http://www.theguardian.com/healthcare-network/2014/dec/18/how-nhs-organisations-should-use-social-media
http://www.theguardian.com/healthcare-network/views-from-the-nhs-frontline/2014/dec/15/nhs-staff-learn-listen
http://gu.com/p/447kg/stw
http://www.theguardian.com/healthcare-network/2014/dec/18/how-nhs-organisations-should-use-social-media
http://bit.ly/1BXqeYo
http://bit.ly/1uYUwGw
http://bit.ly/1z0h4Ej
http://bit.ly/1BXqeYo
http://www.theguardian.com/healthcare-network/2014/dec/17/healthcare-revolution-community-care
http://ow.ly/FYP4S
http://bit.ly/1BXqeYo
https://www.youtube.com/watch?v
http://bit.ly/1AabHXo
http://t.co/d
http://bit.ly/1BXqe

NHS spending inefficiently on non-permanent staff, via @guardian http://gu.com/p/4774n/stw What do you think?


http://gu.com/p/4774n/stw
In case you haven't read it yet: Why is schizophrenia portrayed so negatively on screen? http://gu.com/p/472d5/stw #mentalhealth


http://gu.com/p/472d5/stw
How can technology help reduce mental health waiting times? http://gu.com/p/476ky/stw #NHS #mentalhealth


http://gu.com/p/476ky/stw
Are you a member of the network? Why not sign up? It's free ...  #NHS #healthcare


People with strong relationships are 50% more likely to survive life-threatening illness than those w/ weaker ones. http://gu.com/p/474fe/stw


http://gu.com/p/474fe/stw
The #NHS has to provide therapy to 95% of patients within 18 weeks, but we can do better than this http://gu.com/p/476ky/stw #mentalhealth


http://gu.com/p/476ky/stw
RT @Ieso_Health: Our clinical lead Sarah Bateup is in @GdnHealthcare today discussing how we can help reduce #mentalhealth wait times http:


New today

http://gu.com/p/46efq/stw
'We must, and can, do much more to value HCAs and to invest in them.' http://gu.com/p/46gy5/stw #NHS


http://gu.com/p/46gy5/stw
'The best thing about being a doctor in the NHS today is having insider knowledge.' http://gu.com/p/46d3a/stw #ViewsfromtheNHSfrontline


http://gu.com/p/46d3a/stw
'What being a HCA has taught me is that the foundation for good mental health is relationships.' http://gu.com/p/46gy5/stw #NHS


http://gu.com/p/46gy5/stw
Last night, the 3 biggest political parties set out their plans &amp; answered your questions: http://gu.com/p/46gd2/stw #gdnhealthhustings


http://gu.com/p/46gd2/stw
As healthcare assistants we are often the least trained but most hands-on #NHS staff http://gu.com/p/46gy5/stw


http://gu.com/p/46gy5/stw
We've summed up the 10 best campaigns from #NHSChangeDay – read them here: http://gu.com/p/46fdn/stw


http://gu.com/p/46fdn/stw
'The lesson I have learned is that, as an HCA, I must be hypersensitive to the needs of p

Mental health research: underfunded and failing to attract young academics http://gu.com/p/46vxx/stw via @GdnHigherEd


http://gu.com/p/46vxx/stw
What are the parties' policies on the #NHS? Join our big health debate http://gu.com/p/45p28/stw


http://gu.com/p/45p28/stw
Popular on the network right now: General election 2015: who’s promising what for the NHS? http://gu.com/p/45q2c/stw #NHS #GeneralElection2015


http://gu.com/p/45q2c/stw
Our newsletter is about to go out. Sign up here to receive it for free:  #NHS #healthcare


Don't miss: Compassionate care – a superpower or just part of the job? http://gu.com/p/45p9e/stw


http://gu.com/p/45p9e/stw
New today: General election 2015: who’s promising what for the NHS? by @RichardVize http://gu.com/p/45q2c/stw #NHS #GeneralElection2015


http://gu.com/p/45q2c/stw
Don't miss: Flu crisis: if you want to know what’s really going on, ask your nurses http://gu.com/p/45pqt/stw #NHS


http://gu.com/p/45pqt/stw
In case you missed it y/day: Compa

http://bit.ly/1BDK1wm
Marmot: If a woman tells you a husband has a right to beat his wife if she refuses him sex, how do you react? http://bit.ly/1BDK1wm


http://bit.ly/1BDK1wm
RT @Shaun4Maher: .@GdnHealthcare £25m to treat part of the cause (social care) £700m to treat symptom (NHS failure demand). Wrong-headed? @


This health service of ours is precious, and we need to start caring for it as such. http://bit.ly/1CFRdoE #NHS #ViewsfromtheNHSfrontline


http://bit.ly/1CFRdoE
Abuse leads to premature birth of a woman’s children, sexually transmitted disease and damage to mental health http://bit.ly/1BDK1wm


http://bit.ly/1BDK1wm
Here's the @SocietyGuardian report on last night's #GuardianLiveNHS debate: Minister calls for party unity on NHS http://gu.com/p/453j2/tw


http://gu.com/p/453j2/tw
New: Domestic violence damages health and shortens lives, writes Sir Michael Marmot @WithoutViolence http://bit.ly/1BDK1wm #domesticabuse


http://bit.ly/1BDK1wm
NHS treatment of whistleblowers i

http://gu.com/p/44d3e/stw
.@DHgovuk keen to encourage #newyearhonours nominations from health + care services, and hopes for greater diversity http://gu.com/p/44d3e/stw


http://gu.com/p/44d3e/stw
Popular today: New Year’s Eve in A&amp;E: drunk partygoers add to my workload http://gu.com/p/44cqa/stw


http://gu.com/p/44cqa/stw
How NHS hospitals are preparing for an #Ebola outbreak http://gu.com/p/435qh/stw


http://gu.com/p/435qh/stw
As NYE approaches, @KailashChandOBE warns of the cost of drinking to society and the #NHS http://gu.com/p/44522/stw


http://gu.com/p/44522/stw
As 2014 draws to an end, we look back at some of the network's highlights. Happy New Year! http://gu.com/p/44d5t/stw #NHS #healthcare


http://gu.com/p/44d5t/stw
RT @DrUmeshPrabhu: Don't wait for anyone's permission to do the right thing. Start with #patient safety and staff well-being in your own de


In our top 5 most-read at the moment: This Christmas I’ll be on duty at the hospital where I was born http://gu.co


RT @jsacker: Interesting @Jeremy_Hunt wants to see #CCGs becoming whole person commissioners #reformhealth incl PH, Social Care &amp; 1ry care


RT @DonaghyDan: @Jeremy_Hunt now talks of whole-person commissioning - very similar to @andyburnhammp. Could all be lost on the electorate 


A strong NHS needs a strong economy, says Hunt #reformhealth


Hunt: There will be 3m people in the next couple of years with three long term conditions and highly complex needs #reformhealth


RT @AilsaOfficial: @GdnHealthcare Do you think the "live fast, die old" culture is an issue? Smoking, drinking etc generations of 65+ NHS u


The #ageingpopulation is the biggest single challenge facing the UK, says Jeremy Hunt #reformhealth


Candid #ViewsfromtheNHSfrontline blogpost: I love working in mental health but can’t do a good job on a shoestring http://gu.com/p/43yyb/stw


http://gu.com/p/43yyb/stw
RT @EvolvingPatient: Great to hear @emilpeters highlight that info from health providers is only a fracti

http://theguardian.com/healthcare-net
While charging drunk A&amp;E patients will make life easier for doctors, it is discriminatory says Dr Jason Sarfo-Annin http://gu.com/p/435tf/tw


http://gu.com/p/435tf/tw
Talkpoint: should mental health be given the same priority as physical health? http://gu.com/p/42ydx/tw #NHS #mentalhealth #health


http://gu.com/p/42ydx/tw
Patient data: enabling the future of healthcare 


Data in healthcare: ‘I want to see digital health move into the mainstream’ http://gu.com/p/434ja/tw #NHS #health #data


http://gu.com/p/434ja/tw
Obesity surgery saves NHS cash and improves quality of life, says study http://gu.com/p/436mk/tw #mentalhealth #NHS


http://gu.com/p/436mk/tw
Mental health: Permission to film documentary at Broadmoor was granted after five years of negotiations http://gu.com/p/437z3/tw


http://gu.com/p/437z3/tw
Would you like to write about the highs and lows of working for the NHS? http://gu.com/p/4vkmy/tw #viewsfromtheNHSfrontline


http://gu

Tesco should not be responsible for tackling the UK’s diabetes crisis http://www.theguardian.com/healthcare-network/2014/oct/15/tesco-tackling-diabetes-crisis


http://www.theguardian.com/healthcare-network/2014/oct/15/tesco-tackling-diabetes-crisis
Ebola: Skype consultations could help prevent an outbreak in the UK http://www.theguardian.com/healthcare-network/2014/oct/16/skype-consultations-ebola-outbreak


http://www.theguardian.com/healthcare-network/2014/oct/16/skype-consultations-ebola-outbreak
I love the NHS but stricter rules and tighter budgets make my job difficult  http://www.theguardian.com/healthcare-network/2014/oct/17/love-nhs-rules-less-budget-make-job-difficult


http://www.theguardian.com/healthcare-network/2014/oct/17/love-nhs-rules-less-budget-make-job-difficult
Can Lord Darzi’s plan improve Londoners’ health? http://bit.ly/1wbDdBI


http://bit.ly/1wbDdBI
A day in the life of a specialised musculoskeletal physiotherapist http://bit.ly/1zeE6x3


http://bit.ly/1zeE6x3

In [7]:
frame.head()

,tweetid,date,title,url
0,585876545266286592,Wed Apr 08 18:47:08 +0000 2015,What is palliative care like in India? One GP ...,http://gu.com/p/47a4g/stw
1,585861945535791106,Wed Apr 08 17:49:07 +0000 2015,Most viewed this week: I loved being a midwife...,http://gu.com/p/46zc4/stw
2,585859917350731777,Wed Apr 08 17:41:03 +0000 2015,How can technology improve mental health waiti...,http://gu.com/p/476ky/stw
3,585844465199407104,Wed Apr 08 16:39:39 +0000 2015,In case you missed it: Why the #NHS shouldn’t ...,http://gu.com/p/477tc/stw
4,585800529739001856,Wed Apr 08 13:45:04 +0000 2015,India is a stark example of how commodifying h...,http://gu.com/p/47a4g/stw


In [8]:
dest_url = './run_data/gdnhealth_scrapt_v2.csv'
frame_dest = pd.read_csv(dest_url,  sep='|', encoding='utf-8')
# Write to file txt
file_names = frame['tweetid'].values

for key, name in enumerate(file_names):
    
    # Construct the file name
    located_file = './training/' + str(name) + '.txt'
    
    try:
        # Writing on file
        contents = str(frame_dest['contents'].iloc[key])
        print(key, ' : ' ,len(contents))
        
        if len(contents) > 1:
            # Open file for writing
            f= open(located_file,"w+")

            # Writing content to file
            f.write(contents)
            
            # close file connection
            f.close()
        else:
            print("Index [Out]: ",key)     
    except:
        pass

0  :  4292
1  :  4279
2  :  4091
3  :  4603
4  :  4292
5  :  3
6  :  4603
7  :  3
8  :  4292
9  :  5652
10  :  4091
11  :  4603
12  :  4292
13  :  4603
14  :  3
15  :  6175
16  :  3
17  :  3
18  :  3979
19  :  4603
20  :  3
21  :  4603
22  :  3562
23  :  4603
24  :  3
25  :  4603
26  :  3
27  :  4091
28  :  3
29  :  5244
30  :  3
31  :  4403
32  :  4315
33  :  3967
34  :  4091
35  :  4659
36  :  4352
37  :  3655
38  :  4269
39  :  3655
40  :  3
41  :  4038
42  :  3
43  :  3
44  :  4352
45  :  4622
46  :  1889
47  :  4091
48  :  4315
49  :  4328
50  :  3655
51  :  3693
52  :  4091
53  :  3
54  :  5411
55  :  4315
56  :  3530
57  :  4091
58  :  3
59  :  3655
60  :  4315
61  :  3693
62  :  4352
63  :  4091
64  :  3
65  :  4091
66  :  4352
67  :  3655
68  :  2985
69  :  4315
70  :  4091
71  :  3
72  :  3655
73  :  4091
74  :  3
75  :  4091
76  :  3693
77  :  4352
78  :  4352
79  :  4352
80  :  3655
81  :  4352
82  :  3
83  :  3
84  :  3693
85  :  4352
86  :  4352
87  :  4352
88  :  3898
89

913  :  3
914  :  5573
915  :  3
916  :  3
917  :  4568
918  :  4659
919  :  4659
920  :  4659
921  :  2981
922  :  4659
923  :  4012
924  :  2981
925  :  4659
926  :  4245
927  :  3
928  :  3
929  :  3
930  :  2981
931  :  3
932  :  4659
933  :  4659
934  :  4659
935  :  3
936  :  4659
937  :  3
938  :  4659
939  :  3
940  :  4245
941  :  4245
942  :  4012
943  :  2981
944  :  4767
945  :  2981
946  :  4094
947  :  3
948  :  3
949  :  4767
950  :  4245
951  :  3
952  :  3
953  :  5830
954  :  3942
955  :  3
956  :  5186
957  :  3
958  :  3
959  :  3
960  :  3
961  :  4768
962  :  3
963  :  3
964  :  3
965  :  3
966  :  4245
967  :  5583
968  :  4767
969  :  4012
970  :  3
971  :  4012
972  :  3
973  :  4767
974  :  3942
975  :  4767
976  :  4012
977  :  4012
978  :  4012
979  :  4767
980  :  4767
981  :  4651
982  :  3420
983  :  4767
984  :  3
985  :  4587
986  :  2891
987  :  4767
988  :  4767
989  :  4767
990  :  3
991  :  4767
992  :  3
993  :  3
994  :  4767
995  :  3
996  :  3
9

1706  :  4398
1707  :  4398
1708  :  3531
1709  :  4807
1710  :  3
1711  :  3
1712  :  4398
1713  :  3
1714  :  3
1715  :  3
1716  :  3
1717  :  3531
1718  :  4807
1719  :  3
1720  :  4807
1721  :  3
1722  :  3
1723  :  4807
1724  :  4807
1725  :  4807
1726  :  3
1727  :  3
1728  :  4398
1729  :  3374
1730  :  4807
1731  :  4398
1732  :  3531
1733  :  2746
1734  :  3578
1735  :  3443
1736  :  3531
1737  :  3162
1738  :  1633
1739  :  3403
1740  :  3
1741  :  3531
1742  :  3687
1743  :  3
1744  :  1889
1745  :  3443
1746  :  1633
1747  :  3531
1748  :  3531
1749  :  12855
1750  :  3531
1751  :  3890
1752  :  3890
1753  :  3
1754  :  3531
1755  :  3687
1756  :  3
1757  :  3531
1758  :  3443
1759  :  3687
1760  :  3443
1761  :  3
1762  :  3
1763  :  1889
1764  :  3443
1765  :  3
1766  :  3687
1767  :  3
1768  :  3
1769  :  3443
1770  :  3890
1771  :  6606
1772  :  4750
1773  :  3687
1774  :  3687
1775  :  1633
1776  :  3069
1777  :  3443
1778  :  3
1779  :  4750
1780  :  3443
1781  :  163

2362  :  3954
2363  :  3857
2364  :  3
2365  :  3
2366  :  3954
2367  :  3954
2368  :  2862
2369  :  2862
2370  :  2862
2371  :  4614
2372  :  3857
2373  :  3
2374  :  3
2375  :  3
2376  :  3
2377  :  3
2378  :  3857
2379  :  3
2380  :  3
2381  :  3
2382  :  3
2383  :  3
2384  :  4332
2385  :  3
2386  :  3
2387  :  3
2388  :  3
2389  :  3
2390  :  3857
2391  :  4332
2392  :  4465
2393  :  4332
2394  :  4332
2395  :  3
2396  :  3
2397  :  3
2398  :  4332
2399  :  4465
2400  :  4465
2401  :  4465
2402  :  4465
2403  :  2862
2404  :  4465
2405  :  3820
2406  :  4465
2407  :  3
2408  :  2862
2409  :  4493
2410  :  11250
2411  :  4465
2412  :  3
2413  :  4465
2414  :  4465
2415  :  3
2416  :  4106
2417  :  4465
2418  :  8178
2419  :  3
2420  :  2862
2421  :  1889
2422  :  2862
2423  :  4106
2424  :  4153
2425  :  3
2426  :  3
2427  :  5092
2428  :  2862
2429  :  2862
2430  :  2862
2431  :  5092
2432  :  3899
2433  :  5482
2434  :  4153
2435  :  4499
2436  :  3987
2437  :  4153
2438  :  4312

In [8]:
frame_urls = frame['url'].values

In [9]:
# -------------------------------------------------
# Run for this jupyter notebook
title_attempt = True
for key, url in enumerate(frame_urls):
    try:
        # Start the connection
        res = requests.get(url)        

        # Check return header
        if res.status_code == 200:
            print("Header:  ", 200)
            print(url)
            
            # Initialize XPath
            xpath_selector = Selector(text=res.text)
            
            list_content = dict()
            
            # Scrapt Text with XPath
            contents = xpath_selector.xpath(xpath_bbc[0]).extract()
            contents = join_txt(contents)
            print(len(contents))
#             # Tags
#             tags = xpath_selector.xpath(xpath_bbc[1]).extract()
#             print(len(tags))
            # Author
            author = xpath_selector.xpath(xpath_bbc[2]).extract()
            if len(author):
                author = author[0]
            else:
                author = ''
            print(len(author))
            list_content.update({'contents': contents,'tags': '','author': author})
            
            if title_attempt:
                print("TItle: ", title_attempt)
                df_temp = pd.DataFrame(columns = list(list_content.keys()))
                frame = pd.concat([frame, df_temp])
                print(frame.columns)                
                title_attempt = False
                
            for k_, v_ in list_content.items():
                print(k_," : " ,len(v_))
                frame[k_].iloc[key] = v_
            print("=====================\n")

        else:
            print("['ERRORS'] {} , at {}".format(res.status_code, url))
    except:
        pass
    

Header:   200
http://gu.com/p/47a4g/stw
4292
10
TItle:  True
Index(['author', 'contents', 'date', 'tags', 'title', 'tweetid', 'url'], dtype='object')
contents  :  4292
tags  :  0
author  :  10



scripts.py:38: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  # Find all possible hastags
C:\Users\Shaynie\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Header:   200
http://gu.com/p/46zc4/stw
4279
9
contents  :  4279
tags  :  0
author  :  9

Header:   200
http://gu.com/p/476ky/stw
4091
12
contents  :  4091
tags  :  0
author  :  12

Header:   200
http://gu.com/p/477tc/stw
4603
11
contents  :  4603
tags  :  0
author  :  11

Header:   200
http://gu.com/p/47a4g/stw
4292
10
contents  :  4292
tags  :  0
author  :  10

Header:   200
http://www.theguardian.com/healthcare-network/2015/apr/07/why-nhs-should-not-hog-limelight-general-election
4603
11
contents  :  4603
tags  :  0
author  :  11

Header:   200
http://gu.com/p/47a4g/stw
4292
10
contents  :  4292
tags  :  0
author  :  10

Header:   200
http://gu.com/p/47ajt/stw
5652
14
contents  :  5652
tags  :  0
author  :  14

Header:   200
http://www.theguardian.com/healthcare-network/2015/apr/02/technology-mental-health-waiting-times
4091
12
contents  :  4091
tags  :  0
author  :  12

Header:   200
http://gu.com/p/477tc/stw
4603
11
contents  :  4603
tags  :  0
author  :  11

Header:   200
http://

Header:   200
http://gu.com/p/4732n/stw
3693
13
contents  :  3693
tags  :  0
author  :  13

Header:   200
http://gu.com/p/4732n/stw
3693
13
contents  :  3693
tags  :  0
author  :  13

Header:   200
http://gu.com/p/474fe/stw
3655
15
contents  :  3655
tags  :  0
author  :  15

Header:   200
http://gu.com/p/474fe/stw
3655
15
contents  :  3655
tags  :  0
author  :  15

Header:   200
http://gu.com/p/474fe/stw
3655
15
contents  :  3655
tags  :  0
author  :  15

Header:   200
http://gu.com/p/4732n/stw
3693
13
contents  :  3693
tags  :  0
author  :  13

Header:   200
http://gu.com/p/474fe/stw
3655
15
contents  :  3655
tags  :  0
author  :  15

Header:   200
http://gu.com/p/474fe/stw
3655
15
contents  :  3655
tags  :  0
author  :  15

Header:   200
http://gu.com/p/4732n/stw
3693
13
contents  :  3693
tags  :  0
author  :  13

Header:   200
http://gu.com/p/472bx/stw
4003
0
contents  :  4003
tags  :  0
author  :  0

Header:   200
http://gu.com/p/472d5/stw
4315
17
contents  :  4315
tags  :  0
autho

Header:   200
http://gu.com/p/46qqc/stw
4038
12
contents  :  4038
tags  :  0
author  :  12

Header:   200
http://gu.com/p/46qcf/stw
8581
0
contents  :  8581
tags  :  0
author  :  0

Header:   200
http://gu.com/p/46qqc/stw
4038
12
contents  :  4038
tags  :  0
author  :  12

Header:   200
http://gu.com/p/46qqc/stw
4038
12
contents  :  4038
tags  :  0
author  :  12

['ERRORS'] 404 , at http://t.co/XKCoH
Header:   200
http://www.theguardian.com/healthcare-network/2015/mar/25/nhs-reform-what-next-for-worlds-best-health-system
8581
0
contents  :  8581
tags  :  0
author  :  0

Header:   200
http://bit.ly/19kNUKk
4038
12
contents  :  4038
tags  :  0
author  :  12

Header:   200
http://gu.com/p/46qqc/stw
4038
12
contents  :  4038
tags  :  0
author  :  12

Header:   200
http://gu.com/p/46n8h/stw
4622
1
contents  :  4622
tags  :  0
author  :  1

Header:   200
http://gu.com/p/46nzv/stw
2275
0
contents  :  2275
tags  :  0
author  :  0

Header:   200
http://gu.com/p/46qhj/stw
4097
1
contents  :  409

...

Header:   200
http://bit.ly/1puB5om
2421
10
contents  :  2421
tags  :  0
author  :  10

Header:   200
http://bit.ly/1s8moBW
4863
0
contents  :  4863
tags  :  0
author  :  0

Header:   200
http://bit.ly/1puB5om
2421
10
contents  :  2421
tags  :  0
author  :  10

Header:   200
http://bit.ly/1yXy3sS
3696
1
contents  :  3696
tags  :  0
author  :  1

Header:   200
http://www.theguardian.com/sustainable-business/2014/oct/29/technology-social-innovation-shape-future-healthcare-system-free-event
1464
1
contents  :  1464
tags  :  0
author  :  1

Header:   200
http://bit.ly/1xc4iUO
4541
14
contents  :  4541
tags  :  0
author  :  14

Header:   200
http://bit.ly/1s8moBW
4863
0
contents  :  4863
tags  :  0
author  :  0

Header:   200
http://bit.ly/1xc4iUO
4541
14
contents  :  4541
tags  :  0
author  :  14

Header:   200
http://bit.ly/10igbNq
3103
11
contents  :  3103
tags  :  0
author  :  11

Header:   200
http://bit.ly/10igbNq
3103
11
contents  :  3103
tags  :  0
author  :  11

Header:   200
http:

Header:   200
http://www.theguardian.com/healthcare-network/2014/oct/28/tackle-mental-health-inequality-black-people
4216
1
contents  :  4216
tags  :  0
author  :  1

Header:   200
http://www.theguardian.com/healthcare-network/2014/oct/30/nhs-five-year-forward-view-test-new-models-care
4125
1
contents  :  4125
tags  :  0
author  :  1

Header:   200
http://www.theguardian.com/healthcare-network/2014/oct/30/five-year-forward-review-doesnt-provide-answers
5065
1
contents  :  5065
tags  :  0
author  :  1

Header:   200
http://www.theguardian.com/healthcare-network/2014/oct/29/how-nhs-engage-patients-public
5404
13
contents  :  5404
tags  :  0
author  :  13

Header:   200
http://www.theguardian.com/healthcare-network/2014/oct/30/nhs-five-year-forward-view-test-new-models-care
4125
1
contents  :  4125
tags  :  0
author  :  1

Header:   200
http://www.theguardian.com/healthcare-network/2014/oct/31/clinical-pyschologist-amazed-strength-human-spirit
3757
14
contents  :  3757
tags  :  0
author  

Header:   200
http://bit.ly/1u2cWqT
4216
1
contents  :  4216
tags  :  0
author  :  1

Header:   200
http://bit.ly/101eAuN
5404
13
contents  :  5404
tags  :  0
author  :  13

Header:   200
http://bit.ly/101eAuN
5404
13
contents  :  5404
tags  :  0
author  :  13

Header:   200
http://bit.ly/1u2cWqT
4216
1
contents  :  4216
tags  :  0
author  :  1

Header:   200
http://bit.ly/1u2cWqT
4216
1
contents  :  4216
tags  :  0
author  :  1

Header:   200
http://bit.ly/1u2cWqT
4216
1
contents  :  4216
tags  :  0
author  :  1

Header:   200
http://bit.ly/1wuzbU6
3317
13
contents  :  3317
tags  :  0
author  :  13

Header:   200
http://bit.ly/1wuzbU6
3317
13
contents  :  3317
tags  :  0
author  :  13

Header:   200
http://bit.ly/1u2cWqT
4216
1
contents  :  4216
tags  :  0
author  :  1

Header:   200
http://bit.ly/1u2cWqT
4216
1
contents  :  4216
tags  :  0
author  :  1

Header:   200
http://bit.ly/1wuzbU6
3317
13
contents  :  3317
tags  :  0
author  :  13

Header:   200
http://bit.ly/1u2cWqT
4216
1
c

Header:   200
http://bit.ly/1t6pcES
4385
49
contents  :  4385
tags  :  0
author  :  49

Header:   200
http://bit.ly/1sbq19U
4431
43
contents  :  4431
tags  :  0
author  :  43

Header:   200
http://www.theguardian.com/healthcare-network/2014/oct/23/nhs-five-year-forward-view-challenge-funding
4385
49
contents  :  4385
tags  :  0
author  :  49

Header:   200
http://bit.ly/1t6pcES
4385
49
contents  :  4385
tags  :  0
author  :  49

Header:   200
http://bit.ly/1sbq19U
4431
43
contents  :  4431
tags  :  0
author  :  43

Header:   200
http://gu.com/p/42ky3/tw
6862
22
contents  :  6862
tags  :  0
author  :  22

Header:   200
http://bit.ly/ZEj8Y3
4042
60
contents  :  4042
tags  :  0
author  :  60

Header:   200
http://bit.ly/1zkdlYf
3843
57
contents  :  3843
tags  :  0
author  :  57

Header:   200
http://bit.ly/1pwhKO5
4673
1
contents  :  4673
tags  :  0
author  :  1

Header:   200
http://bit.ly/1sFoNr6
4315
16
contents  :  4315
tags  :  0
author  :  16

Header:   200
http://bit.ly/1pzZNy6
834

Header:   200
http://bit.ly/1w19DNe
4452
15
contents  :  4452
tags  :  0
author  :  15

Header:   200
http://bit.ly/1wbDdBI
3776
1
contents  :  3776
tags  :  0
author  :  1

Header:   200
http://bit.ly/1zeE6x3
3227
9
contents  :  3227
tags  :  0
author  :  9



In [10]:
frame.head()

,author,contents,date,tags,title,tweetid,url
0,Hannah Fox,"hen you step out of the NHS for a year, as I h...",Wed Apr 08 18:47:08 +0000 2015,,What is palliative care like in India? One GP ...,5.858765e+17,http://gu.com/p/47a4g/stw
1,Anonymous,I am a midwife with eight years experience and...,Wed Apr 08 17:49:07 +0000 2015,,Most viewed this week: I loved being a midwife...,5.858619e+17,http://gu.com/p/46zc4/stw
2,Sarah Bateup,veryone working in mental health will welcome ...,Wed Apr 08 17:41:03 +0000 2015,,How can technology improve mental health waiti...,5.858599e+17,http://gu.com/p/476ky/stw
3,Louise Bond,he NHS is set to take centre stage as the gene...,Wed Apr 08 16:39:39 +0000 2015,,In case you missed it: Why the #NHS shouldn’t ...,5.858445e+17,http://gu.com/p/477tc/stw
4,Hannah Fox,"hen you step out of the NHS for a year, as I h...",Wed Apr 08 13:45:04 +0000 2015,,India is a stark example of how commodifying h...,5.858005e+17,http://gu.com/p/47a4g/stw


In [13]:
frame.shape

(2993, 7)

In [14]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2993 entries, 0 to 2992
Data columns (total 7 columns):
author      2173 non-null object
contents    2173 non-null object
date        2993 non-null object
tags        2173 non-null object
title       2993 non-null object
tweetid     2993 non-null float64
url         2993 non-null object
dtypes: float64(1), object(6)
memory usage: 187.1+ KB


In [12]:
frame.to_csv('./run_data/gdnhealth_scrapt_v2.csv', sep='|', encoding='utf-8', index=False)